In [ ]:
import json
from os import listdir

In [ ]:
## ABRIR O ULTIMO DB

megasena_db_dir = './db/megasena/'
dbs = listdir(megasena_db_dir)
filename = megasena_db_dir+dbs[-1]
with open(filename, 'r') as openfile: 
    db = json.load(openfile)

In [ ]:
# O formato da chave é "Concurso NNN (dd/mm/aaaa)"
def find_concurso(dica, db):
    found_key = None
    for key in db.keys():
        if str(dica) in key:
            found_key = key
    if found_key == None:
        found_key = "Concurso não encontrado."
    return found_key

# def find_ultimo_concurso(db):
#     i = 0
#     ultimo_concurso = 'Concurso '+str(i)
#     find_concurso(ultimo_concurso)
#     while find_concurso(ultimo_concurso, db) not in 'Concurso não encontrado.'
    
def find_buraco(db):
    concurso = 1
    faltantes = []
    while concurso <= 2246:
        if find_concurso('Concurso '+str(concurso)+' ', db) in 'Concurso não encontrado.':
            faltantes.append(concurso)
        concurso+=1
    return faltantes


faltantes = find_buraco(db)
print('\nNão foram encontrados os concursos:', faltantes, '\n\nUm total de:', len(faltantes))

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import json
import time
from selenium.webdriver.chrome.options import Options
opts = Options()
opts.add_argument("user-agent=[Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36]")

browser = webdriver.Chrome()

browser.get("http://loterias.caixa.gov.br/wps/portal/loterias/landing/megasena/") 
time.sleep(5)

resultados = {}

for faltante in faltantes:
    busca = browser.find_element_by_xpath('//*[@id="buscaConcurso"]')
    busca.send_keys(Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE, Keys.BACK_SPACE)
    busca.send_keys(faltante)
    if ' '+str(faltante)+' ' not in (browser.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span')).text:
        busca.send_keys(Keys.ENTER)
        time.sleep(1)
    concurso = (browser.find_element_by_xpath('//*[@id="conteudoresultado"]/div[1]/div/h2/span')).text
    numeros = []
    numeros.append((browser.find_element_by_xpath('//*[@id="ulDezenas"]')).text)
    resultado_numeros = []
    while len(numeros[0]) > 0:
        resultado_numeros.append(int(numeros[0][:2]))
        numeros[0] = numeros[0][2:]
    numeros = resultado_numeros
    resultados[concurso] = {
        'Numeros': numeros
    }
    print(concurso, numeros)
print('Done.')

In [ ]:
## SALVAR NOVO DB
db.update(resultados)

megasena_db_dir = './db/megasena/megasena_'
dictionary = db
filename = megasena_db_dir+str(int(time.time()))+'.json'
with open(filename, "w") as outfile: 
    json.dump(dictionary, outfile)